# Planning Lab

In [ ]:
!pip install --pre unified-planning==1.0.0.95.dev1
!pip install up_fast_downward==0.3.1

In [ ]:
!pip install matplotlib==3.7.1  # for visualisation in this notebook

In [ ]:
from unified_planning.shortcuts import *

import unified_planning as up

up.shortcuts.get_environment().credits_stream = None

## Part (a): Model the Task

The code below models a simpler version of the Household Robot domain with two rooms and one open door between them. The robot is in room A initially and needs to move to room B. You can use it as a starting point for your own solution to the full Household Robot task.

In [ ]:
def get_planning_task():
    # Declare user types.
    Room = UserType("Room")
    Door = UserType("Door")
    Key = UserType("Key")

    # Declare predicates.
    robot_in = up.model.Fluent("robot_in", BoolType(), r=Room)
    connected = up.model.Fluent("connected", BoolType(), r1=Room, d=Door, r2=Room)
    room_has_door = up.model.Fluent("room_has_door", BoolType(), r=Room, d=Door)
    room_has_key = up.model.Fluent("room_has_key", BoolType(), r=Room, k=Key)
    key_opens_door = up.model.Fluent("key_opens_door", BoolType(), d=Door, k=Key)
    robot_has_key = up.model.Fluent("robot_has_key", BoolType(), k=Key)
    door_is_unlocked = up.model.Fluent("door_is_unlocked", BoolType(), d=Door)

    # Add (typed) objects to problem.
    problem = up.model.Problem("household")

    def get_room(room):
        return up.model.Object(f"room{room}", Room)

    def get_door(door):
        return up.model.Object(f"door{door}", Door)

    def get_key(key):
        return up.model.Object(f"key{key}", Key)

    roomLiving = get_room("Living")
    roomCorridor = get_room("Corridor")
    roomLobby = get_room("Lobby")
    roomBath = get_room("Bath")
    roomKitchen = get_room("Kitchen")
    rooms = [roomLiving, roomCorridor, roomLobby, roomBath, roomKitchen]

    doorL = get_door("L")
    doorC = get_door("C")
    doorB = get_door("B")
    doorF = get_door("F")
    doorK = get_door("K")

    doors = [doorL, doorC, doorB, doorF, doorK]

    keyL = get_key("L")
    keyC = get_key("C")
    keyB = get_key("B")
    keyF = get_key("F")
    keyK = get_key("K")
    keyAll = get_key("All")
    keys = [keyL, keyC, keyB, keyF, keyK, keyAll]

    problem.add_objects(rooms)
    problem.add_objects(doors)
    problem.add_objects(keys)

    connections = [
        (roomLiving, doorL, roomCorridor),
        (roomCorridor, doorC, roomLobby),
        (roomCorridor, doorB, roomBath),
        (roomLiving, doorK, roomKitchen)
    ]

    rooms_and_doors = []
    for room1, door, room2 in connections:
        rooms_and_doors.append((room1, door))
        rooms_and_doors.append((room2, door))

    rooms_and_doors.append((roomLobby, doorF))

    # Specify the initial state.
    problem.add_fluent(robot_in, default_initial_value=False)
    problem.add_fluent(connected, default_initial_value=False)
    problem.add_fluent(room_has_door, default_initial_value=False)
    problem.add_fluent(room_has_key, default_initial_value=False)
    problem.add_fluent(key_opens_door, default_initial_value=False)
    problem.add_fluent(robot_has_key, default_initial_value=False)
    problem.add_fluent(door_is_unlocked, default_initial_value=False)
    problem.set_initial_value(robot_in(roomLiving), True)

    for room1, door, room2 in connections:
        problem.set_initial_value(connected(room1, door, room2), True)
        problem.set_initial_value(connected(room2, door, room1), True)

    for room, door in rooms_and_doors:
        problem.set_initial_value(room_has_door(room, door), True)

    problem.set_initial_value(room_has_key(roomLiving, keyL), True)
    problem.set_initial_value(room_has_key(roomLiving, keyK), True)
    problem.set_initial_value(room_has_key(roomBath, keyC), True)
    problem.set_initial_value(room_has_key(roomBath, keyF), True)
    problem.set_initial_value(room_has_key(roomCorridor, keyB), True)
    problem.set_initial_value(room_has_key(roomKitchen, keyAll), True)

    problem.set_initial_value(key_opens_door(doorL, keyL), True)
    problem.set_initial_value(key_opens_door(doorC, keyC), True)
    problem.set_initial_value(key_opens_door(doorB, keyB), True)
    problem.set_initial_value(key_opens_door(doorF, keyF), True)
    problem.set_initial_value(key_opens_door(doorK, keyK), True)

    for door in doors:
        problem.set_initial_value(key_opens_door(door, keyAll), True)

    # Add actions.
    move = up.model.InstantaneousAction("move", room1=Room, door=Door, room2=Room)
    room1 = move.parameter("room1")
    door = move.parameter("door")
    room2 = move.parameter("room2")
    move.add_precondition(robot_in(room1))
    move.add_precondition(connected(room1, door, room2))
    move.add_precondition(door_is_unlocked(door))
    move.add_effect(robot_in(room1), False)
    move.add_effect(robot_in(room2), True)
    problem.add_action(move)

    open_door = up.model.InstantaneousAction("open_door", room=Room, door=Door, key=Key)
    door = open_door.parameter("door")
    key = open_door.parameter("key")
    room = open_door.parameter("room")
    open_door.add_precondition(robot_in(room))
    open_door.add_precondition(room_has_door(room, door))
    open_door.add_precondition(robot_has_key(key))
    open_door.add_precondition(key_opens_door(door, key))
    open_door.add_effect(door_is_unlocked(door), True)
    problem.add_action(open_door)

    pick_up_key = up.model.InstantaneousAction("pick_up_key", room=Room, key=Key)
    room = pick_up_key.parameter("room")
    key = pick_up_key.parameter("key")
    pick_up_key.add_precondition(robot_in(room))
    pick_up_key.add_precondition(room_has_key(room, key))
    pick_up_key.add_effect(robot_has_key(key), True)
    pick_up_key.add_effect(room_has_key(room, key), False)
    problem.add_action(pick_up_key)

    # Specify the goal.
    problem.add_goal(robot_in(roomLobby))
    problem.add_goal(door_is_unlocked(doorF))

    # We want to minimize the plan cost.
    problem.add_quality_metric(MinimizeActionCosts({}, default=Int(1)))
    return problem

problem = get_planning_task()


## Part (b): Find a (possibly suboptimal) plan

Solve the task with greedy best-first search using the FF heuristic. The example code below uses the h^add heuristic. You need to inspect the output to stdout to see the heuristic value of the initial state.

In [ ]:
params = {
    "fast_downward_search_config": "let(hff,ff(),eager_greedy([hff],preferred=[hff]))"
}

with OneshotPlanner(name="fast-downward", params=params) as planner:
    result = planner.solve(problem, output_stream=sys.stdout)

    if result.status == up.engines.PlanGenerationResultStatus.SOLVED_SATISFICING:
        print("Found a plan of length:", len(result.plan.actions))
        print(result.plan)

        with PlanValidator() as validator:
            val_result = validator.validate(problem, result.plan)
            print("Plan cost:", val_result.metric_evaluations)

    else:
        print("No plan found.")

INFO     planner time limit: None
INFO     planner memory limit: None

INFO     Running translator.
INFO     translator stdin: None
INFO     translator time limit: None
INFO     translator memory limit: None
INFO     translator command line string: /usr/bin/python3 /usr/local/lib/python3.10/dist-packages/up_fast_downward/downward/builds/release/bin/translate/translate.py /tmp/tmp2joy2sk9/domain.pddl /tmp/tmp2joy2sk9/problem.pddl --sas-file output.sas
Parsing...
Parsing: [0.000s CPU, 0.002s wall-clock]
Normalizing task... [0.000s CPU, 0.000s wall-clock]
Instantiating...
Generating Datalog program... [0.000s CPU, 0.001s wall-clock]
Normalizing Datalog program...
Normalizing Datalog program: [0.000s CPU, 0.001s wall-clock]
Preparing model... [0.000s CPU, 0.001s wall-clock]
Generated 18 rules.
Computing model... [0.000s CPU, 0.002s wall-clock]
130 relevant atoms
76 auxiliary atoms
206 final queue length
223 total queue pushes
Completing instantiation... [0.000s CPU, 0.001s wall-clock]
Inst

## Part (c): Find an optimal plan

Solve the task with A* using the `iPDB` heuristic.

In [ ]:
params = {
    "fast_downward_search_config": "astar(ipdb())"
}

with OneshotPlanner(name="fast-downward", params=params) as planner:
    result = planner.solve(problem, output_stream=sys.stdout)

    if result.status == up.engines.PlanGenerationResultStatus.SOLVED_SATISFICING:
        print("Found a plan of length:", len(result.plan.actions))
        print(result.plan)

        with PlanValidator() as validator:
            val_result = validator.validate(problem, result.plan)
            print("Plan cost:", val_result.metric_evaluations)

    else:
        print("No plan found.")

INFO     planner time limit: None
INFO     planner memory limit: None

INFO     Running translator.
INFO     translator stdin: None
INFO     translator time limit: None
INFO     translator memory limit: None
INFO     translator command line string: /usr/bin/python3 /usr/local/lib/python3.10/dist-packages/up_fast_downward/downward/builds/release/bin/translate/translate.py /tmp/tmpyznbppbb/domain.pddl /tmp/tmpyznbppbb/problem.pddl --sas-file output.sas
Parsing...
Parsing: [0.010s CPU, 0.003s wall-clock]
Normalizing task... [0.000s CPU, 0.000s wall-clock]
Instantiating...
Generating Datalog program... [0.000s CPU, 0.001s wall-clock]
Normalizing Datalog program...
Normalizing Datalog program: [0.000s CPU, 0.001s wall-clock]
Preparing model... [0.000s CPU, 0.001s wall-clock]
Generated 18 rules.
Computing model... [0.000s CPU, 0.002s wall-clock]
130 relevant atoms
76 auxiliary atoms
206 final queue length
223 total queue pushes
Completing instantiation... [0.000s CPU, 0.001s wall-clock]
Inst